# YOUTH 1 SUB WALLET

## 1. Initialise a controller for Yoma Youth Sub Wallet 1

In [21]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://yoma-multi-tenant-agent:8021"

# JWT KEY for Youth 1 Sub Wallet
%store -r youth1_token

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, jwt_token=youth1_token)

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f4bacba24f0>


IPython autoawait is `on`, and set to use `asyncio`


## 2. Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [5]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)


Task exception was never retrieved
future: <Task finished name='Task-7' coro=<AriesAgentController.listen_webhooks() done, defined at /aries_basic_controller/aries_controller.py:140> exception=OSError(98, "error while attempting to bind on address ('0.0.0.0', 8022): address already in use")>
Traceback (most recent call last):
  File "/aries_basic_controller/aries_controller.py", line 147, in listen_webhooks
    await self.webhook_site.start()
  File "/opt/conda/lib/python3.8/site-packages/aiohttp/web_runner.py", line 121, in start
    self._server = await loop.create_server(
  File "/opt/conda/lib/python3.8/asyncio/base_events.py", line 1463, in create_server
    raise OSError(err.errno, 'error while attempting '
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8022): address already in use


## 8a. Check for any existing connections

In [6]:
# Check for existing connections
connection = await agent_controller.connections.get_connections()
print("EXISTING CONNECTIONS")
for key, value in connection.items():
    for item in value:
        print('ConnectionID:', item['connection_id'], 'Status:',item['state'])

EXISTING CONNECTIONS


### 8. Paste the invitation from the Alice notebook into the invitation variable here

In [7]:
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '93a80613-1a43-4b25-8600-ccdd9a3777d5', 'label': 'EXTERNAL', 'recipientKeys': ['Dp5tKBouwB36p4p4VvgG1HuSc2qmayHXtFiXWogaWYLH'], 'serviceEndpoint': 'https://24feee5e7d3a.ngrok.io'}

### 9. Accept the invitation, then move to Alice's notebook

In [8]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invitation)
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
connection_id = response["connection_id"]
STATE = response["state"]

Connection {'connection_id': '54d5ec42-3ff3-4388-b03b-8d7229b77820', 'their_role': 'inviter', 'their_label': 'EXTERNAL', 'request_id': '118cffc5-c52c-471b-9427-5de3079a7b07', 'state': 'request', 'invitation_mode': 'once', 'invitation_key': 'Dp5tKBouwB36p4p4VvgG1HuSc2qmayHXtFiXWogaWYLH', 'accept': 'manual', 'my_did': 'Fmi9Wy2tav1kgDTftj7CPz', 'updated_at': '2021-03-11 19:06:55.434345Z', 'created_at': '2021-03-11 19:06:55.363052Z', 'rfc23_state': 'request-sent', 'routing_state': 'none'}


### 13. Check if connection state is active

It should be if you send a trust ping from Alice's side

In [9]:
# Print connection list
connection = await agent_controller.connections.get_connection(connection_id)
print("Youth Wallet 1 AGENT CONNECTION")
print(connection)
print("State:", connection["state"])

Youth Wallet 1 AGENT CONNECTION
{'connection_id': '54d5ec42-3ff3-4388-b03b-8d7229b77820', 'their_role': 'inviter', 'their_label': 'EXTERNAL', 'request_id': '118cffc5-c52c-471b-9427-5de3079a7b07', 'state': 'active', 'invitation_mode': 'once', 'invitation_key': 'Dp5tKBouwB36p4p4VvgG1HuSc2qmayHXtFiXWogaWYLH', 'accept': 'manual', 'my_did': 'Fmi9Wy2tav1kgDTftj7CPz', 'updated_at': '2021-03-11 19:07:00.719104Z', 'created_at': '2021-03-11 19:06:55.363052Z', 'rfc23_state': 'completed', 'their_did': 'Eipg4W4719TFTJAuCXyj5N', 'routing_state': 'none'}
State: active


## 8d. Check if established connection is in active state

In [10]:
import time

# print('Current state for ConnectionId {} is {}'.format(connection_id,STATE))
print(colored("Current state for ConnectionId {} is {}".format(connection_id,STATE), "magenta", attrs=["bold"]))
while STATE != 'active':
#     print('ConnectionId {0} is not in active state yet'.format(connection_id))
    print(colored("ConnectionId {0} is not in active state yet".format(connection_id), "yellow", attrs=["bold"]))
    trust_ping = await agent_controller.messaging.trust_ping(connection_id,'hello!')
#     print('Trust ping send to ConnectionId {0} to activate connection'.format(trust_ping))
    print(colored("Trust ping send to ConnectionId {0} to activate connection".format(trust_ping), "blue", attrs=["bold"]))
    time.sleep(5)
    
# print('ConnectionId: {0} is now active. Continue with notebook'.format(connection_id))
print(colored("ConnectionId: {0} is now active. Continue with notebook".format(connection_id), "green", attrs=["bold"]))


Current state for ConnectionId 54d5ec42-3ff3-4388-b03b-8d7229b77820 is request
ConnectionId 54d5ec42-3ff3-4388-b03b-8d7229b77820 is not in active state yet
Trust ping send to ConnectionId {'thread_id': '9b4418d1-ed87-4043-9462-c869e97c4b4a'} to activate connection
ConnectionId 54d5ec42-3ff3-4388-b03b-8d7229b77820 is not in active state yet
Trust ping send to ConnectionId {'thread_id': '3300a506-db2c-4233-b535-876451243e60'} to activate connection
ConnectionId 54d5ec42-3ff3-4388-b03b-8d7229b77820 is not in active state yet
Trust ping send to ConnectionId {'thread_id': '76b9a9c8-a19a-4c1c-9923-c76bfebb346c'} to activate connection
ConnectionId 54d5ec42-3ff3-4388-b03b-8d7229b77820 is not in active state yet
Trust ping send to ConnectionId {'thread_id': 'e0cb17f7-ab85-4684-9e56-1a3d74f93d2e'} to activate connection
ConnectionId 54d5ec42-3ff3-4388-b03b-8d7229b77820 is not in active state yet
Trust ping send to ConnectionId {'thread_id': 'e425e561-521e-4dd1-abae-0ec78fcd1049'} to activate co

CancelledError: 

## 10. Check Credential Exchange Records

The agent will have at least one record if you have run through the issuer notebook up until send credential.


In [24]:
response = await agent_controller.issuer.get_records()
results = response["results"]
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")



Credential exchange 0dc0115d-615e-4a60-b86b-d00464abfc42, role: holder, state: credential_acked
Being offered: [{'name': 'fullname', 'value': 'Lohan Spies'}, {'name': 'mobile', 'value': '0831234567'}, {'name': 'email', 'value': 't@t.com'}]


## 12. Store the credential

Once the issuer has responded to a request by sending the credential, the holder needs to store it to save the credential for later.

First check that the credential record is in the credential_received state


In [23]:
record = await agent_controller.issuer.get_record_by_id(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Credential exchange 0dc0115d-615e-4a60-b86b-d00464abfc42, role: holder, state: credential_acked


## 11. End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [20]:
response = await agent_controller.terminate()
print(response)

None


# Proceed to Part 4 on [Verifier Notebook](http://localhost:8889/notebooks/Part%204%20-%20Verify%20a%20Presentation.ipynb)

Here you will present the attributes issued to you within this tutorial to a verifying entity.